# A Zipf classifier
## Using Zipf and statistics to map books to their authors and periods

## Introduction
Hello, I'm Luca Cappelletti and here I will show you an example of usage of [ZipfClassifier](https://github.com/LucaCappelletti94/zipf_classifier), a classifier that leverages the assumption that some kind of datasets (texts, [some images](http://www.dcs.warwick.ac.uk/bmvc2007/proceedings/CD-ROM/papers/paper-288.pdf), even [sounds in spoken languages](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0033993)) follows the [Zipf distribution](https://en.wikipedia.org/wiki/Zipf%27s_law).

## How to use this notebook

This is a [Jupyter Notebook](http://jupyter.org/). You can either read it [here on github](https://github.com/LucaCappelletti94/zipf_classifier/blob/master/Classifying%20authors.ipynb) or, **preferably** to enjoy all its aspects, run it on your own computer. Jupyter comes installed with [Anaconda](https://anaconda.org/), to execute it you just need to run the following in your terminal:

`jupyter-notebook`

## What we will use

### The packages
We will use obviously the [ZipfClassifier](https://github.com/LucaCappelletti94/zipf_classifier) and other two packages of mine: [Zipf](https://github.com/LucaCappelletti94/zipf) to create the distributions from the texts and [Dictances](https://github.com/LucaCappelletti94/dictances) for the classifications metrics. If you need to install them just run the following command in your terminal:

```pip install zipf dictances zipf_classifier```

In [1]:
from zipf.factories import ZipfFromDir
from zipf_classifier import ZipfClassifier
from dictances import jensen_shannon, normal_total_variation, kullback_leibler, bhattacharyya, bhattacharyya_coefficient, hellinger

### Additional packages
We will also be using some utilities, such as the loading bar `tqdm` and the `requests` package. If you don't have them already you can install them by running:

```
pip install tqdm requests tabulate
```

The others packages should be already installed with python by default.

In [1]:
import io
import inspect
import json
import math
import os
import random
import shutil
import zipfile
from collections import defaultdict
from pprint import pprint

import requests
from tqdm import tqdm_notebook as tqdm

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML, display
import tabulate

In [2]:
txts = [sub[0]+'/'+txt for sub in os.walk('recipes') for txt in sub[2] if txt.endswith('.txt')]
for txt in tqdm(txts):
    n = txt.count('/')
    if n == 2:
        #print(txt)
        target = txt[:-4]
        #print("Creating %s"%target)
        shutil.move(txt, target)

HBox(children=(IntProgress(value=0, max=31986), HTML(value='')))

### Some small helpers
Let's make ome small functions to help out loading folders:

In [3]:
def get_dirs(root):
    """Return subdirectories under a directory."""
    return [root+"/"+d for d in os.listdir(root) if os.path.isdir(root+"/"+d)]

and the book folders:

In [4]:
def get_books(root):
    """Return all books found under a given root."""
    return [book[0] for book in os.walk(root) for chapter in book[2][:1] if chapter.endswith('.txt')]

and the saved zipfs:

In [5]:
def get_zipfs(root):
    """Return all zipfs found under a given root."""
    return [zipfs[0]+"/"+zipf for zipfs in os.walk(root) for zipf in zipfs[2] if zipf.endswith('.json')]

### Some small helpers

In [6]:
def b(string):
    """Return a boldified string."""
    return "\033[1m%s\033[0;0m"%string

In [7]:
def red(string):
    """Return a red string."""
    return "\033[0;31m%s\033[0;0m"%string

In [8]:
def yellow(string):
    """Return a yellow string."""
    return "\033[0;33m%s\033[0;0m"%string

In [9]:
def green(string):
    """Return a green string."""
    return "\033[0;32m%s\033[0;0m"%string

In [10]:
def success(results, metric):
    """Show the result of a given test."""
    successes = results["success"]
    total = successes + results["failures"] + results["unclassified"]
    percentage = round(successes/total*100,2)
    if percentage > 85:
        metric_name = green(metric.__name__)
    elif percentage > 70:
        metric_name = yellow(metric.__name__)
    else:
        metric_name = red(metric.__name__)
    print("Success with metric %s: %s"%(metric_name,b(str(percentage)+"%")))
    display(HTML(tabulate.tabulate(list(results.items()), tablefmt='html')))

In [11]:
def print_function(function):
    """Print the source of a given function."""
    code = inspect.getsource(function)
    formatter = HtmlFormatter()
    display(HTML('<style type="text/css">{}</style>{}'.format(
        formatter.get_style_defs('.highlight'),
        highlight(code, PythonLexer(), formatter))))

### The datasets
I've prepared a **datasets** of old famous authors (Mary Shelley, Dumas, Carroll...), available in the repo and downloadable [here](https://github.com/LucaCappelletti94/zipf_classifier/blob/master/dataset.zip?raw=true): the authors are separeted by period and their books are already split into chapters, with extension `.txt`.

#### Retrieving the datasets
We download and extract the dataset:

In [12]:
datasets = {
    "authors": [
        "authors.zip"
    ],
    "periods": [
        "periods.1.zip",
        "periods.2.zip",
        "periods.3.zip"
    ],
    "recipes": [
        "recipes.1.zip",
        "recipes.2.zip"
    ]
}

In [13]:
def download_dataset(filename):
    """Download a dataset into current folder showing progressbar."""
    if os.path.exists(filename):
        return None
    url = "https://github.com/LucaCappelletti94/zipf_classifier/blob/master/%s?raw=true"%filename
    print("Downloading %s from %s"%(b(filename), url))
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0)); 
    block_size = 1024  * 1024
    wrote = 0 
    with open(filename, 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='MB', unit_scale=True):
            wrote = wrote  + len(data)
            f.write(data)
    if total_size != 0 and wrote != total_size:
        return False
    return True

In [14]:
def extract_zip(path, name):
    """Extract zip file at given path."""
    print("Extracting %s"%b(path))
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(name)
    zip_ref.close()

In [15]:
for name, chunks in datasets.items():
    for chunk in chunks:
        result = download_dataset(chunk)
        if result:
            extract_zip(chunk, name)
        elif result is None:
            print("Dataset %s component %s was already downloaded."%(b(name), green(chunk)))
        else:
            print("Had problems downloading %s for dataset %s"%(red(chunk), b(name)))
print("Done!")

Dataset authors component authors.zip was already downloaded.
Dataset periods component periods.1.zip was already downloaded.
Dataset periods component periods.2.zip was already downloaded.
Dataset periods component periods.3.zip was already downloaded.
Done!


#### Splitting into train and test
Let's say we leave 70% to learning and 30% to testing. Let's proceed to split the dataset in two:

In [16]:
learning_percentage = 0.6

First we check if the dataset is already split (this might be a re-run):

In [17]:
def is_already_split(root):
    """Return a bool indicating if the dataset has already been split."""
    split_warns = ["learning", "testing"]
    for sub_dir in os.listdir(root):
        for split_warn in split_warns:
            if split_warn in sub_dir:
                return True
    return False

Then we split the dataset's books as follows:

Since we want the zipfs that the classifier will use to do the classification built on a proportioned dataset, we pick the percentage of books put aside for learning from the minimum number of books for class in the dataset.

In [18]:
def split_books(root, percentage):
    """Split the dataset into learning and testing."""
    min_books = math.inf
    for book_class in get_dirs(root):
        books = get_books(book_class)
        min_books = min(min_books, len(books))
    for book_class in get_dirs(root):
        books = get_books(book_class)
        random.seed(42) # for reproducibility
        random.shuffle(books) # Shuffling books
        n = int(min_books*percentage)
        learning_set, testing_set = books[:n], books[n:] # splitting books into the two partitions
        # Moving into respective folders
        [shutil.copytree(book, "%s/learning/%s"%(root, book[len(root)+1:])) for book in learning_set]
        [shutil.copytree(book, "%s/testing/%s"%(root, book[len(root)+1:])) for book in testing_set]

Here we actually run the two functions:

In [19]:
for dataset in datasets:
    if is_already_split(dataset):
        print("I believe I've already split the dataset %s!"%(b(dataset)))
    else:
        split_books(dataset, learning_percentage)

### The metrics

After experimental analysis I've determined that the [Jensen Shannon Divergence](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence) is one of the best metric for this kind of classification, other being the [Hellinger distance](https://en.wikipedia.org/wiki/Hellinger_distance) and the [Bhattacharyya distance](https://en.wikipedia.org/wiki/Bhattacharyya_distance). 

_A small test to experimentally verify this claim is run in the end of the presentation._

#### Reasons for using these distances
1. They are defined on distributions that do not necessarily share all events.
2. They can be implemented with computational complexity $O(\min(n,m))$, where $n$ and $m$ are respectively the length of two distributions $P, Q$ when one assumes that the $\sum_{i\in P}^n p_i = 1$ and $\sum_{i\in Q}^m q_i = 1$.

### Kullback Leibler Divergence
$$
    D_{KL}(P,Q) = \sum_i P(i) \log{\frac{P(i)}{Q(i)}}
$$
The KL divergece is defined for all events in a set $P, Q \subseteq X$. 

This forces to define the KL for zipfs only on the subset of the events that are shared beetween the two distributions: $X = P \cap Q$.

This ignores all the information about non-sharec events and it is solved via the Jensen Shannon divergence.

In [20]:
print_function(kullback_leibler)

### Jensen Shannon Divergence
$$
    JSD(P,Q) = \frac{1}{2}D(P,M) + \frac{1}{2}D(Q, M) \qquad M = \frac{1}{2}(P+Q)
$$

The KL divergence is defined for every event in a set $X=P\cup Q$, it is **symmetric** and has **always a finite value**.

#### Getting the current implementation

The current implementation works as follows:

Starting from the extended formulation:

$$
    m_i = \frac{1}{2}(p_i+q_i), \quad p_i = \begin{cases}
        p_i & i \in P\\
        0 & otherwise
    \end{cases}, \quad q_i = \begin{cases}
        q_i & i \in Q\\
        0 & otherwise
    \end{cases}
$$

$$
    JSD(P,Q) = \frac{1}{2}\sum_{i \in P} p_i \log{\frac{p_i}{m_i}} + \frac{1}{2}\sum_{j \in Q} q_j \log{\frac{q_j}{m_j}}
$$

Replacing in the formulation $m_i$:

$$
    JSD(P,Q) = \frac{1}{2}\sum_{i \in P} p_i \log{\frac{p_i}{\frac{1}{2}(p_i+q_i)}} + \frac{1}{2}\sum_{j \in Q} q_j \log{\frac{q_j}{\frac{1}{2}(p_j+q_j)}}
$$

Splitting the sums in 3 parts: $i \in P\setminus P\cap Q$, $i \in P\cap Q$ and $i \in Q\setminus P\cap Q$.

$$
    JSD(P,Q) = JSD_1(P,Q) + JSD_2(P,Q) + JSD_3(P,Q)
$$

\begin{align}
    JSD_1(P,Q) &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{\frac{p_i}{\frac{1}{2}(p_i+q_i)}} + \frac{1}{2}\sum_{j \in P\setminus P\cap Q} q_j \log{\frac{q_j}{\frac{1}{2}(p_j+q_j)}}\\
               &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{\frac{p_i}{\frac{1}{2}(p_i+q_i)}}\\
               &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{\frac{p_i}{\frac{1}{2}p_i}}\\
               &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{\frac{1}{\frac{1}{2}}}\\
               &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{2}\\
               &= \frac{1}{2}\log{2}\sum_{i \in P\setminus P\cap Q} p_i\\
\end{align}

\begin{align}
    JSD_2(P,Q) &= \frac{1}{2}\sum_{i \in P\setminus P\cap Q} p_i \log{\frac{p_i}{\frac{1}{2}(p_i+q_i)}} + \frac{1}{2}\sum_{j \in P\setminus P\cap Q} q_j \log{\frac{q_j}{\frac{1}{2}(p_j+q_j)}}\\
               &= \frac{1}{2}\sum_{i \in P\cap Q} p_i \log{\frac{p_i}{\frac{1}{2}(p_i+q_i)}} + q_i \log{\frac{q_i}{\frac{1}{2}(p_i+q_i)}}\\
               &= \frac{1}{2}\sum_{i \in P\cap Q} p_i \log{\frac{2p_i}{p_i+q_i}} + q_i \log{\frac{2q_i}{p_i+q_i}}\\
\end{align}

\begin{align}
    JSD_3(P,Q) &= \frac{1}{2}\log{2}\sum_{j \in Q\setminus P\cap Q} q_j\\
\end{align}

Summing $JSD_1$ and $JSD_3$ we can obtain:

$JSD_1+JSD_3 = \frac{1}{2}\log{2}\left(\sum_{i \in P\setminus P\cap Q} p_i + \sum_{j \in Q\setminus P\cap Q} q_j\right)$

In particular, if $\sum_{j\in Q}^m q_j = 1$ and $\sum_{i\in P}^n p_i = 1$, we can write:

\begin{align}
JSD_1+JSD_3 &= \frac{1}{2}\log{2}\left(2 - \sum_{i \in P\cap Q} p_i - \sum_{j \in P\cap Q} q_j\right)\\
            &= \frac{1}{2}\log{2}\left(2 - \sum_{i \in P\cap Q} p_i + q_j\right)\\
\end{align}

Putting all togheter we obtain:

$JSD(P,Q) =  \frac{1}{2}\left[\sum_{i \in P\cap Q} \left(p_i \log{\frac{2p_i}{p_i+q_i}} + q_i \log{\frac{2q_i}{p_i+q_i}}\right) + \log{2}\left(2 - \sum_{i \in P\cap Q} p_i + q_j\right)\right]$

What's marvelous about this semplification is that the computational complexity decrease from a naive literal interpretation of the initial formula of $O(n+m)$ to $O(\min(n,m))$ simply choosing to iterate over whichever of the two distributions holds less events.

The process is nearly identical for all other metrics shown below:

In [21]:
print_function(jensen_shannon)

In [22]:
print_function(normal_total_variation)

In [23]:
print_function(bhattacharyya_coefficient)

In [24]:
print_function(bhattacharyya)

In [25]:
print_function(hellinger)

### The options
We will use the following options for learning and testing. More informations about options' customizations is available [here](https://github.com/LucaCappelletti94/zipf).

In [26]:
options = {}

## Creating the Zipfs
We will now convert all the chapters in the dataset into the respective zipf for each option.

In [27]:
def create_zipfs(paths, factory, test_path):
    for data_path in tqdm(paths, unit=' zipf'):
        path = "%s/%s.json"%(test_path, '/'.join(data_path.split('/')[1:]))
        # If the zipf already exists we skip it
        if os.path.exists(path):
            continue
        path_dirs = '/'.join(path.split('/')[:-1])
        zipf = factory.run(data_path, ['txt'])
        if not zipf.is_empty():
            if not os.path.exists(path_dirs):
                os.makedirs(path_dirs)
            zipf.save(path)

We define the paths for zipfs and their sources:

In [28]:
def get_build_paths(dataset):
    """Return a triple with the build paths for given dataset."""
    learning_path = "%s/learning"%dataset
    testing_path = "%s/testing"%dataset
    zipfs_path = '%s/zipfs'%dataset

    print("I will build learning zipfs from %s,\ntesting zipfs from %s\nand save them in %s\n"%(b(learning_path), b(testing_path), b(zipfs_path)))
    return learning_path, testing_path, zipfs_path

First we create the learning zipfs:

In [29]:
def build_learning_zipfs(path, zipfs_path):
    """Build zipfs from txt files at given path."""
    print("Creating learning zipfs in %s"%(b(path)))
    book_classes = get_dirs(path)
    print("Some of the paths I'm converting are:")
    random.seed(42) # For reproducibility
    random.shuffle(book_classes)
    pprint(book_classes[:10])
    create_zipfs(book_classes, factory, zipfs_path)

And then the testing zipfs:

In [30]:
def build_testing_zipfs(path, zipfs_path):
    """Build zipfs from txt files at given path."""
    print("Creating testing zipfs in %s"%(b(path)))
    books = get_books(path)
    print("Some of the paths I'm converting are:")
    random.seed(42) # For reproducibility
    random.shuffle(books)
    pprint(books[:10])
    create_zipfs(books, factory, zipfs_path)

We create a factory for creating the zipfs objects from files with the options defined above. More informations about factory customization and other possible factories is available [here](https://github.com/LucaCappelletti94/zipf).

In [31]:
factory = ZipfFromDir(options=options)
print("Created a factory with options %s"%(factory))

Created a factory with options {
  "remove_stop_words": false,
  "stop_words": "it",
  "minimum_count": 0,
  "chain_min_len": 1,
  "chain_max_len": 1,
  "chaining_character": " ",
  "sort": false
}


Wake up zipfs factory daemons:

In [32]:
factory.start_processes()

Actually creating the zipfs:

In [33]:
for dataset in datasets:
    print("Building dataset %s"%(b(dataset)))
    learning_path, testing_path, zipfs_path = get_build_paths(dataset)
    build_learning_zipfs(learning_path, zipfs_path)
    build_testing_zipfs(testing_path, zipfs_path)

Building dataset authors
I will build learning zipfs from authors/learning,
testing zipfs from authors/testing
and save them in authors/zipfs

Creating learning zipfs in authors/learning
Some of the paths I'm converting are:
['authors/learning/twain',
 'authors/learning/dh_lawrence',
 'authors/learning/wilde']


HBox(children=(IntProgress(value=0, max=3), HTML(value='')))


Creating testing zipfs in authors/testing
Some of the paths I'm converting are:
['authors/testing/twain/3275',
 'authors/testing/twain/320',
 'authors/testing/wilde/florentine-tragedy',
 'authors/testing/dh_lawrence/4483',
 'authors/testing/wilde/2252',
 'authors/testing/twain/3297',
 'authors/testing/wilde/2305',
 'authors/testing/dh_lawrence/fantasia-of-unconscious',
 'authors/testing/wilde/2317',
 'authors/testing/twain/3259']


HBox(children=(IntProgress(value=0, max=160), HTML(value='')))


Building dataset periods
I will build learning zipfs from periods/learning,
testing zipfs from periods/testing
and save them in periods/zipfs

Creating learning zipfs in periods/learning
Some of the paths I'm converting are:
['periods/learning/romanticism',
 'periods/learning/realism',
 'periods/learning/naturalism',
 'periods/learning/modernism']


HBox(children=(IntProgress(value=0, max=4), HTML(value='')))


Creating testing zipfs in periods/testing
Some of the paths I'm converting are:
['periods/testing/romanticism/579',
 'periods/testing/modernism/3452',
 'periods/testing/romanticism/550',
 'periods/testing/romanticism/2124',
 'periods/testing/romanticism/4545',
 'periods/testing/romanticism/491',
 'periods/testing/modernism/3484',
 'periods/testing/romanticism/143',
 'periods/testing/modernism/blanco-posnet',
 'periods/testing/realism/indian-summer']


HBox(children=(IntProgress(value=0, max=864), HTML(value='')))

Slaying daemons:

In [34]:
factory.close_processes()


## Creating the Classifier
Now we have rendered the learning. Let's run some tests!

In [35]:
def get_classifier_paths(dataset):
    """Return paths for classifier, given a dataset."""
    zipfs_path = get_build_paths(dataset)[2]
    learning_zipfs_path = "%s/learning"%zipfs_path
    testing_zipfs_path = "%s/testing"%zipfs_path
    return learning_zipfs_path, testing_zipfs_path

In [36]:
def load_zipfs(classifier, path):
    """Load zipfs from given path into given classifier."""
    print("Loading zipfs from %s"%(b(path)))
    loaded = []
    for zipf in tqdm(get_zipfs(path)):
        book_class = zipf.split('/')[-1].split('.')[0]
        args = zipf, book_class
        loaded.append(args)
        classifier.add_zipf(*args)
    
    print("Some of the loaded zipfs and its class:")
    for path, cls in loaded[:10]:
        print("Path: %s, class: %s"%(b(path), b(cls)))

In [37]:
def load_test_couples(path):
    """Return list of zipfs from given path."""
    print("Loading tests from %s"%(b(path)))
    test_couples = []
    for zipf in tqdm(get_zipfs(path)):
        book_class = zipf.split('/')[-2]
        args = zipf, book_class
        test_couples.append(args)

    print("Some of the loaded zipfs and its class:")
    random.seed(42)
    random.shuffle(test_couples)
    for path, cls in test_couples[:10]:
        print("Path: %s, class: %s"%(b(path), b(cls)))
    return test_couples

In [38]:
def metrics_test(classifier, test_couples):
    """Run test on all metrics usable on zipfs."""
    metrics = [hellinger]#[normal_total_variation, kullback_leibler, bhattacharyya, hellinger, jensen_shannon]
    for metric in tqdm(metrics, unit='metric'):
        results = classifier.test(test_couples, metric)
        success(results, metric)

First we create the classifier with the options set above:

In [39]:
classifier = ZipfClassifier(options)
print("We're using a classifier with options %s"%classifier)

We're using a classifier with options {
    "sort": false
}


In [40]:
for dataset in datasets:
    print("Testing dataset %s"%(b(dataset)))
    learning_zipfs_path, testing_zipfs_path = get_classifier_paths(dataset)
    load_zipfs(classifier, learning_zipfs_path)
    test_couples = load_test_couples(testing_zipfs_path)
    metrics_test(classifier, test_couples)
    classifier.clear()

Testing dataset authors
I will build learning zipfs from authors/learning,
testing zipfs from authors/testing
and save them in authors/zipfs

Loading zipfs from authors/zipfs/learning


HBox(children=(IntProgress(value=0, max=3), HTML(value='')))


Some of the loaded zipfs and its class:
Path: authors/zipfs/learning/twain.json, class: twain
Path: authors/zipfs/learning/dh_lawrence.json, class: dh_lawrence
Path: authors/zipfs/learning/wilde.json, class: wilde
Loading tests from authors/zipfs/testing


HBox(children=(IntProgress(value=0, max=160), HTML(value='')))


Some of the loaded zipfs and its class:
Path: authors/zipfs/testing/twain/double-barrelled.json, class: twain
Path: authors/zipfs/testing/twain/3294.json, class: twain
Path: authors/zipfs/testing/wilde/2280.json, class: wilde
Path: authors/zipfs/testing/dh_lawrence/3484.json, class: dh_lawrence
Path: authors/zipfs/testing/wilde/2299.json, class: wilde
Path: authors/zipfs/testing/twain/3277.json, class: twain
Path: authors/zipfs/testing/wilde/2318.json, class: wilde
Path: authors/zipfs/testing/dh_lawrence/3487.json, class: dh_lawrence
Path: authors/zipfs/testing/wilde/2288.json, class: wilde
Path: authors/zipfs/testing/twain/323.json, class: twain


...


Some of the loaded zipfs and its class:
Path: periods/zipfs/testing/romanticism/680.json, class: romanticism
Path: periods/zipfs/testing/modernism/sea-and-sardinia.json, class: modernism
Path: periods/zipfs/testing/romanticism/158.json, class: romanticism
Path: periods/zipfs/testing/romanticism/fugitive-pieces.json, class: romanticism
Path: periods/zipfs/testing/romanticism/3882.json, class: romanticism
Path: periods/zipfs/testing/romanticism/129.json, class: romanticism
Path: periods/zipfs/testing/modernism/misalliance.json, class: modernism
Path: periods/zipfs/testing/romanticism/684.json, class: romanticism
Path: periods/zipfs/testing/modernism/in-the-cage.json, class: modernism
Path: periods/zipfs/testing/realism/4137.json, class: realism


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

Done testing 100.000000%

Success with metric hellinger: 79.51%


success,687
failures,177
unclassified,0
mean_delta,0.0132782
Mistook Modernism for Naturalism,24
Mistook Realism for Romanticism,44
Mistook Realism for Naturalism,10
Mistook Modernism for Realism,7
Mistook Modernism for Romanticism,36
Mistook Romanticism for Naturalism,4
Mistook Naturalism for Modernism,11
